In [1]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [10]:
# start mt5
mt5.initialize()

True

In [66]:
# retrieving historical data
rates = mt5.copy_rates_range('EURUSD', mt5.TIMEFRAME_H1, datetime(2023, 9, 1), datetime.now())
df = pd.DataFrame(rates)[['time', 'open', 'high', 'low', 'close']]
df['time'] = pd.to_datetime(df['time'], unit='s')
df

,time,open,high,low,close
0,2023-08-31 22:00:00,1.08458,1.08506,1.08424,1.08434
1,2023-08-31 23:00:00,1.08438,1.08470,1.08414,1.08438
2,2023-09-01 00:00:00,1.08428,1.08433,1.08375,1.08415
3,2023-09-01 01:00:00,1.08420,1.08446,1.08409,1.08420
4,2023-09-01 02:00:00,1.08420,1.08452,1.08418,1.08423
...,...,...,...,...,...
1486,2023-11-27 20:00:00,1.09344,1.09425,1.09322,1.09420
1487,2023-11-27 21:00:00,1.09423,1.09566,1.09422,1.09561
1488,2023-11-27 22:00:00,1.09561,1.09574,1.09506,1.09568
1489,2023-11-27 23:00:00,1.09569,1.09569,1.09520,1.09539


In [67]:
# calculating indicators
df['sma_10'] = df['close'].rolling(10).mean().shift(1)  # shifting 1 because we're working with previous close
df['sma_100'] = df['close'].rolling(100).mean().shift(1)

df['sma_diff'] = df['sma_10'] - df['sma_100']  # if sma_diff > 0, fast_sma is above slow_sma
df['prev_sma_diff'] = df['sma_diff'].shift(1)  # to check previous value for crossover. if sma_diff > 0 and prev_sma_diff < 0, it is a bullish crossover

df['atr'] = (df['high'] - df['low']).rolling(14).mean().shift(1)  # calculating ATR to use for SL and TP

df = df.dropna()
df

,time,open,high,low,close,sma_10,sma_100,sma_diff,prev_sma_diff,atr
101,2023-09-07 03:00:00,1.07238,1.07259,1.07190,1.07222,1.072213,1.077468,-0.005255,-0.005238,0.000938
102,2023-09-07 04:00:00,1.07222,1.07310,1.07201,1.07310,1.072345,1.077346,-0.005001,-0.005255,0.000904
103,2023-09-07 05:00:00,1.07309,1.07310,1.07231,1.07278,1.072525,1.077236,-0.004711,-0.005001,0.000908
104,2023-09-07 06:00:00,1.07279,1.07318,1.07247,1.07300,1.072595,1.077122,-0.004527,-0.004711,0.000883
105,2023-09-07 07:00:00,1.07300,1.07304,1.07178,1.07191,1.072615,1.077009,-0.004394,-0.004527,0.000826
...,...,...,...,...,...,...,...,...,...,...
1486,2023-11-27 20:00:00,1.09344,1.09425,1.09322,1.09420,1.094250,1.091576,0.002674,0.002794,0.001109
1487,2023-11-27 21:00:00,1.09423,1.09566,1.09422,1.09561,1.094240,1.091565,0.002675,0.002674,0.001125
1488,2023-11-27 22:00:00,1.09561,1.09574,1.09506,1.09568,1.094317,1.091587,0.002730,0.002675,0.001183
1489,2023-11-27 23:00:00,1.09569,1.09569,1.09520,1.09539,1.094337,1.091620,0.002717,0.002730,0.001174


In [68]:
fig = px.line(df, x='time', y=['close', 'sma_10', 'sma_100'])
fig

In [69]:
# signal function 1
def get_signal1(data):
    "Long SMA Crossover"

    # bullish crossover
    if data['sma_diff'] > 0 and data['prev_sma_diff'] < 0:
        return 1

    else:
        return 0

In [70]:
# signal function 2
def get_signal2(data):
    "Short SMA Crossover"

    # bullish crossover
    if data['sma_diff'] < 0 and data['prev_sma_diff'] > 0:
        return -1

    else:
        return 0

In [76]:
# setting up DataFrame that will contain backtest trades
trades = pd.DataFrame(columns=['state', 'order_type', 'open_time', 'open_price', 'close_time', 'close_price', 'sl', 'tp'])

for i in df.index:
    data = df.loc[i]

    # specify different signal functions in this block
    signal = get_signal1(data)
    # signal = get_signal2(data)

    # open trade logic
    if signal == 1:
        sl = data['open'] - 3*data['atr']
        tp = data['open'] + 3*data['atr']
        trades.loc[len(trades), trades.columns] = ['open', 'buy', data['time'], data['close'], None, None, sl, tp]
    elif signal == - 1:
        sl = data['open'] + 3*data['atr']
        tp = data['open'] - 3*data['atr']
        trades.loc[len(trades), trades.columns] = ['open', 'sell', data['time'], data['close'], None, None, sl, tp]

    # close trade logic
    open_trades = trades[trades['state'] == 'open']

    # if there are no open trades, skip close position logic
    if open_trades.empty:
        continue

    # close positions that hit sl or tp, iterating though open trades with index x
    for x in open_trades.index:
        t = open_trades.loc[x]
        if t['order_type'] == 'buy':
            if t['sl'] >= data['low']:
                # filling exactly at SL price might cause inaccuracy in backtest as you will receive slippage many times.
                trades.loc[x, ['state', 'close_time', 'close_price']] = ['closed', data['time'], t['sl']]
            elif t['tp'] <= data['high']:
                trades.loc[x, ['state', 'close_time', 'close_price']] = ['closed', data['time'], t['tp']]

        elif t['order_type'] == 'sell':
            if t['sl'] <= data['high']:
                # filling exactly at SL price might cause inaccuracy in backtest as you will receive slippage many times.
                trades.loc[x, ['state', 'close_time', 'close_price']] = ['closed', data['time'], t['sl']]
            elif t['tp'] >= data['low']:
                trades.loc[x, ['state', 'close_time', 'close_price']] = ['closed', data['time'], t['tp']]

    # used for closing trades at the end of backtest
    last_time = data['time']
    last_close = data['close']

# after backtest is over, close all open positions
trades.loc[trades['state']=='open', ['state', 'close_time', 'close_price']] = ['closed', last_time, last_close]

trades

,state,order_type,open_time,open_price,close_time,close_price,sl,tp
0,closed,buy,2023-09-11 10:00:00,1.0726,2023-09-12 04:00:00,1.076111,1.070429,1.076111
1,closed,buy,2023-09-12 22:00:00,1.07282,2023-09-14 15:00:00,1.069511,1.069511,1.076969
2,closed,buy,2023-09-19 15:00:00,1.06995,2023-09-19 18:00:00,1.068214,1.068214,1.073566
3,closed,buy,2023-09-29 08:00:00,1.05764,2023-09-29 11:00:00,1.06118,1.05482,1.06118
4,closed,buy,2023-10-05 11:00:00,1.05041,2023-10-05 19:00:00,1.053895,1.047865,1.053895
5,closed,buy,2023-10-17 20:00:00,1.05672,2023-10-18 19:00:00,1.052366,1.052366,1.063514
6,closed,buy,2023-10-19 17:00:00,1.0569,2023-10-19 19:00:00,1.060706,1.054354,1.060706
7,closed,buy,2023-10-30 16:00:00,1.06096,2023-10-31 11:00:00,1.06415,1.05725,1.06415
8,closed,buy,2023-11-02 07:00:00,1.05962,2023-11-02 15:00:00,1.064601,1.055399,1.064601
9,closed,buy,2023-11-08 23:00:00,1.07088,2023-11-09 21:00:00,1.067038,1.067038,1.074902


In [77]:
# backtest parameters
volume = 100000  # 1 lot

# simplified calc profit function. Needs modifications when it comes to non-USD assets
def calc_profit(x):
    if x['order_type'] == 'buy':
        return (x['close_price'] - x['open_price']) * volume
    elif x['order_type'] == 'sell':
        return (x['open_price'] - x['close_price']) * volume

trades['profit'] = trades.apply(calc_profit, axis=1).round(2)
trades['profit_cumulative'] = trades['profit'].cumsum()
trades

,state,order_type,open_time,open_price,close_time,close_price,sl,tp,profit,profit_cumulative
0,closed,buy,2023-09-11 10:00:00,1.0726,2023-09-12 04:00:00,1.076111,1.070429,1.076111,351.14,351.14
1,closed,buy,2023-09-12 22:00:00,1.07282,2023-09-14 15:00:00,1.069511,1.069511,1.076969,-330.86,20.28
2,closed,buy,2023-09-19 15:00:00,1.06995,2023-09-19 18:00:00,1.068214,1.068214,1.073566,-173.64,-153.36
3,closed,buy,2023-09-29 08:00:00,1.05764,2023-09-29 11:00:00,1.06118,1.05482,1.06118,354.00,200.64
4,closed,buy,2023-10-05 11:00:00,1.05041,2023-10-05 19:00:00,1.053895,1.047865,1.053895,348.50,549.14
5,closed,buy,2023-10-17 20:00:00,1.05672,2023-10-18 19:00:00,1.052366,1.052366,1.063514,-435.36,113.78
6,closed,buy,2023-10-19 17:00:00,1.0569,2023-10-19 19:00:00,1.060706,1.054354,1.060706,380.57,494.35
7,closed,buy,2023-10-30 16:00:00,1.06096,2023-10-31 11:00:00,1.06415,1.05725,1.06415,319.00,813.35
8,closed,buy,2023-11-02 07:00:00,1.05962,2023-11-02 15:00:00,1.064601,1.055399,1.064601,498.07,1311.42
9,closed,buy,2023-11-08 23:00:00,1.07088,2023-11-09 21:00:00,1.067038,1.067038,1.074902,-384.21,927.21


In [78]:
fig2 = px.line(trades, x='open_time', y='profit_cumulative', title='Cumulative Profit')
fig2

In [79]:
# visualize backtest

fig3 = go.Figure(data=[go.Candlestick(
                name='GBPJPY',
                x=df['time'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])

fig3.update_layout(xaxis_rangeslider_visible=False, height=600)

for i, trade in trades.tail(30).iterrows():

    color = 'green' if trade['profit'] > 0 else 'red'
    fig3.add_shape(type="line",
        x0=trade['open_time'], y0=trade['open_price'], x1=trade['close_time'], y1=trade['close_price'],
        line=dict(
            color=color,
            width=2,
            dash="dot",
        )
    )

fig3